# Atelier 1 - Abaque de Gy

Ce notebook permet de visualiser les lignes d'isocontours de l'écart-type relatif de Gy.

Vous pouvez entrer vos propres paramètres, les valider, et obtenir une procédure graphique adaptée à vos étapes.

---

## ✅ Étapes :

1. Entrer vos paramètres globaux :  
- **$a_L$** : proportion massique du lot analysé  
- **$\delta_a$**, **$\delta_g$** : masse spécifique du constituant d’intérêt et masse spécifique de la gangue 
- **$d_0$** : taille à laquelle le constituant d’intérêt est entièrement libéré (en cm)  
- **$M_L$** : masse totale du lot (en g)  
- **s_vals** : liste des écarts-types relatifs souhaités (ex. `[0.01, 0.002]`)

2. Ajouter une ou plusieurs étapes avec leurs paramètres spécifiques :  
- **$M_e$** : masse d’échantillon (en g)  
- **$M_L$** : masse totale à l’étape (en g)  
- **$d$** : taille max des fragments (en cm)

3. Visualiser l’abaque mis à jour avec :  
- les lignes d’isocontours en noir pour les écarts-types choisis  
- les points rouges représentant chaque étape saisie  
- les flèches bleues indiquant la progression entre étapes  
- le calcul et l’affichage de l’écart-type relatif global ($s_r$ global)

---

## 📊 Exemple de sortie :

L’abaque affiche une échelle logarithmique en abscisse (taille des fragments, cm) et en ordonnée (masse de l’échantillon, g).  
Les courbes noires sont les isocontours pour les écarts-types relatifs choisis, tandis que les courbes en gris pointillé correspondent à au maillage logarithmique.

Chaque point rouge correspond à une étape saisie, avec sa valeur sr affichée en rouge à côté.  
Les flèches bleues montrent le cheminement entre étapes.  

Enfin, le $s_r$ global (écart-type total combiné) est indiqué en bleu en haut à gauche.

---

> **Remarque** : les calculs reposent sur les paramètres et formules spécifiques à la géotechnique minière.  
> Assurez-vous de bien ajuster les valeurs selon votre contexte d’étude.


In [36]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from ipywidgets import VBox, HBox, Button, FloatText, Output, Label, Layout, GridBox
from IPython.display import display, clear_output

# --- Paramètres globaux ---
params = {
    'al': 0.010 / 0.67,
    'da': 4.1,
    'dg': 2.8,
    'd0': 0.1
}

common_layout = Layout(width='225px')
common_style = {'description_width': 'initial'}

# Inputs globaux
al_input = FloatText(description='$a_L$ (conc.)', value=params['al'], step=0.01,
                     layout=common_layout, style=common_style)
da_input = FloatText(description='$\delta_a$ (densité min.)', value=params['da'], step=0.1,
                     layout=common_layout, style=common_style)
dg_input = FloatText(description='$\delta_g$ (densité gangue)', value=params['dg'], step=0.1,
                     layout=common_layout, style=common_style)
d0_input = FloatText(description='$d_0$ (taille lib.)', value=params['d0'], step=0.0001,
                     layout=common_layout, style=common_style)

params_widgets = VBox([al_input, da_input, dg_input, d0_input])

# --- Paramètres sr désiré ---
sr_desire_input = FloatText(value=10.0, step=0.1, layout=Layout(width='100px'))
validation_label = Label()
status_label = Label(value='')
error_label = Label(value='', layout=Layout(margin='10px 0 0 0', color='red'))

# --- Fonctions Gy ---
def gy(al, da, dg, ml, d0, me=None, d=None, sr=None, f=0.5, g=0.25):
    if d is None:
        fl = 1.0
    else:
        fl = min(np.sqrt(d0 / d), 1.0)
    ud = (1 - al) / al * ((1 - al) * da + al * dg)
    k = ud * f * g * fl
    if me is None:
        ime = sr**2 / k / d**3 + 1 / ml
        sr = 1 / ime
    elif d is None:
        for _ in range(10):
            d3 = sr**2 / k / (1 / me - 1 / ml)
            d = d3 ** (1 / 3)
            fl = min(np.sqrt(d0 / d), 1.0)
            k = ud * f * g * fl
        sr = d
    else:
        s2 = k * d**3 / me * (1 - me / ml)
        sr = np.sqrt(s2)
    return sr

# --- Étapes Me / Ml / d ---
steps = []
gridbox_container = VBox()
output = Output()

def refresh_gridbox():
    all_widgets = []
    for me, ml, d in steps:
        all_widgets.extend([me, ml, d])
    global steps_grid
    steps_grid = GridBox(all_widgets,
                         layout=Layout(
                             grid_template_columns="150px 150px 150px",
                             grid_gap="10px"
                         ))
    gridbox_container.children = [steps_grid]

def add_step(_=None):
    me_input = FloatText(description='$M_e$ (g)', value=100.0, step=1.0,
                         layout=Layout(width='150px'), style=common_style)
    ml_input = FloatText(description='$M_L$ (g)', value=1000.0, step=1.0,
                         layout=Layout(width='150px'), style=common_style)
    d_input = FloatText(description='$d$ (cm)', value=0.1, step=0.01,
                        layout=Layout(width='150px'), style=common_style)
    steps.append((me_input, ml_input, d_input))
    refresh_gridbox()

def remove_step(_=None):
    if steps:
        steps.pop()
        refresh_gridbox()

def update_params(_=None):
    try:
        params['al'] = float(al_input.value)
        params['da'] = float(da_input.value)
        params['dg'] = float(dg_input.value)
        params['d0'] = float(d0_input.value)
    except Exception as e:
        print("Erreur de saisie dans les paramètres globaux:", e)

# --- Fonction pour tracer les contours Gy ---
def plot_gy_iso_contours(ax, al, da, dg, d0, ml, s_vals=[0.05, 0.1, 0.2]):
    f, g = 0.5, 0.25
    d = np.logspace(-2, 1, 200)  # tailles de fragments cm
    flib = np.minimum(1, np.sqrt(d0 / d))
    ud = (1 - al) / al * ((1 - al) * da + al * dg)
    k = ud * f * g * flib * d**3

    for s in s_vals:
        me_vals = 1 / (s**2 / k + 1 / ml)
        ax.loglog(d, me_vals, 'k', linewidth=1.2)
        i = np.argmin(np.abs(me_vals - 50))
        if i < len(d):
            ax.text(d[i]*1.3, me_vals[i], f'{s*100:.0f}%', fontsize=10, rotation=45)

# --- Calcul et tracé ---
def update_plot(_=None):
    with output:
        clear_output(wait=True)
        error_label.value = ''
        update_params()

        sr_list = []
        valid_coords = []

        for idx, (me_input, ml_input, d_input) in enumerate(steps):
            me = me_input.value
            ml = ml_input.value
            d = d_input.value
            if me <= 0 or ml <= 0 or d <= 0:
                continue
            sr = gy(params['al'], params['da'], params['dg'], ml, params['d0'], me, d)
            sr_list.append(sr)
            valid_coords.append((d, me))

        if not sr_list:
            print("Aucune étape valide.")
            return

        sr_global = np.sqrt(np.sum(np.array(sr_list) ** 2))

        # Tracé
        fig, ax = plt.subplots(figsize=(8,5))

        # --- Lignes iso Gy ---
        ml_values = [ml_input.value for _, ml_input, _ in steps]
        me_values = [me_input.value for me_input, _, _ in steps]
        d_values = [d_input.value for _, _, d_input in steps]
        ml_moyen = np.max(ml_values) if ml_values else 1000
        #plot_gy_iso_contours(ax, params['al'], params['da'], params['dg'], params['d0'], ml_moyen,
        #             s_vals=[0.01, 0.05, 0.1, 0.5])

        # --- Ligne sr désiré en rouge ---
        f, g = 0.5, 0.25
        d = np.logspace(-2, 1, 200)
        flib = np.minimum(1, np.sqrt(params['d0'] / d))
        ud = (1 - params['al']) / params['al'] * ((1 - params['al']) * params['da'] + params['al'] * params['dg'])
        k = ud * f * g * flib * d**3
        sr_desire = sr_desire_input.value / 100
        me_vals_desire = 1 / (sr_desire**2 / k + 1 / ml_moyen)
        ax.loglog(d, me_vals_desire, 'r', linewidth=2, label=f'sr désiré = {sr_desire*100:.1f}%')
        ax.legend()

        # Étapes
        for i, (d, me) in enumerate(valid_coords):
            ax.plot(d, me, 'ro')
            ax.text(d*1.1, me*1.1, f'{sr_list[i]*100:.2f}%', fontsize=9, color='red')

        # Flèches
        for i in range(len(valid_coords)-1):
            x1, y1 = valid_coords[i]
            x2, y2 = valid_coords[i+1]
            ax.add_patch(FancyArrowPatch((x1, y1), (x2, y1), arrowstyle='->', color='blue', mutation_scale=12, lw=1.5))
            ax.add_patch(FancyArrowPatch((x2, y1), (x2, y2), arrowstyle='->', color='blue', mutation_scale=12, lw=1.5))

        ax.set_xlabel('Taille max fragments d (cm)')
        ax.set_ylabel('Masse échantillon me (g)')
        ax.set_xscale('log')
        ax.set_yscale('log')
        if len(steps) == 1:
            ax.set_xlim(5e-3, 1.1 * max(d_values))
            ax.set_ylim(10, 1.1 * max(ml_values))
        else:
            ax.set_xlim(0.9 * min(d_values), 1.1 * max(d_values))
            ax.set_ylim(0.9 * min(me_values), 1.1 * max(ml_values))
        ax.grid(True, which='both')
        ax.set_title('Abaque de Gy - Écarts-types relatifs par étape')
        plt.show()

        # Validation
        sr_desire = sr_desire_input.value / 100
        if sr_global <= sr_desire:
            validation_label.value = f"✅ Procédure valide (sr_global = {sr_global*100:.2f}% ≤ sr désiré = {sr_desire*100:.2f}%)"
        else:
            validation_label.value = f"❌ Procédure NON valide (sr_global = {sr_global*100:.2f}% > sr désiré = {sr_desire*100:.2f}%)"

# --- Boutons ---
btn_add = Button(description='Ajouter étape', button_style='success')
btn_add.on_click(add_step)

btn_remove = Button(description='Supprimer dernière étape', button_style='warning')
btn_remove.on_click(remove_step)

btn_calcul = Button(description='Calculer', button_style='primary')
btn_calcul.on_click(update_plot)

# Observateurs
for w in [al_input, da_input, dg_input, d0_input]:
    w.observe(update_params, names='value')

# --- Interface ---
ui = VBox([
    Label("Paramètres globaux :"),
    params_widgets,
    HBox([Label("$s_r$ désiré (%) :"), sr_desire_input]),
    HBox([btn_add, btn_remove, btn_calcul]),
    gridbox_container,
    status_label,
    validation_label,
    error_label,
    output
])

display(ui)

